# RFM 分析

使用 UCI Online Retail 資料集

* [https://archive.ics.uci.edu/ml/datasets/Online+Retail](https://archive.ics.uci.edu/ml/datasets/Online+Retail)

參考資料：

* https://en.wikipedia.org/wiki/RFM_(market_research)

## 下載資料集

In [1]:
# 在colab中可以用這個
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx

--2022-08-20 05:31:00--  https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23715344 (23M) [application/x-httpd-php]
Saving to: ‘Online Retail.xlsx’

Online Retail.xlsx  100%[===================>]  22.62M  55.4MB/s    in 0.4s    

2022-08-20 05:31:00 (55.4 MB/s) - ‘Online Retail.xlsx’ saved [23715344/23715344]



In [2]:
# 在windows上，用這個試試看
!curl -o "Online Retail.xlsx" https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22.6M  100 22.6M    0     0  40.5M      0 --:--:-- --:--:-- --:--:-- 40.4M


## 讀取資料集

In [3]:
import pandas as pd
import numpy as np

In [4]:
Retail_df = pd.read_excel("Online Retail.xlsx")

In [5]:
Retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
Retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [7]:
Retail_df['amount'] = Retail_df['Quantity']*Retail_df['UnitPrice']

In [8]:
Retail_df.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.3


In [9]:
Retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
 8   amount       541909 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 37.2+ MB


In [10]:
Retail_df['InvoiceDate'].min()

Timestamp('2010-12-01 08:26:00')

In [11]:
Retail_df['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

## 刪除缺失值

### 刪除CustomerID缺失值 (方法一)

In [12]:
df1 = Retail_df.dropna(subset=["CustomerID"])

In [13]:
len(df1)

406829

### 刪除CustomerID缺失值 (方法二)

In [14]:
df = Retail_df[Retail_df['CustomerID'].notnull()]

In [15]:
ref_date = df['InvoiceDate'].max()

## 計算 R, F, M 值

In [19]:
# 計算最近一次購買時間
recency = df['InvoiceDate'].groupby(df['CustomerID']).max()
# 計算購買的次數
frequency = df['InvoiceDate'].groupby(df['CustomerID']).count()
# 計算購買的金額
monetary = df['amount'].groupby(df['CustomerID']).sum()

In [21]:
# 計算上一次購買間隔天數(幾天前)
r_interval = (ref_date - recency).dt.days

# pandas cut 切分

In [22]:
# 計算 R, F, M 值
r_score = pd.cut(r_interval, 5, labels=[5, 4, 3, 2, 1])
f_score = pd.cut(frequency, 5, labels=[1, 2, 3, 4, 5])
m_score = pd.cut(monetary, 5, labels=[1, 2, 3, 4, 5])

In [35]:
r_score

CustomerID
12346.0    1
12347.0    5
12348.0    5
12349.0    5
12350.0    1
          ..
18280.0    2
18281.0    3
18282.0    5
18283.0    5
18287.0    5
Name: InvoiceDate, Length: 4372, dtype: category
Categories (5, int64): [5 < 4 < 3 < 2 < 1]

In [36]:
f_score

CustomerID
12346.0    1
12347.0    1
12348.0    1
12349.0    1
12350.0    1
          ..
18280.0    1
18281.0    1
18282.0    1
18283.0    1
18287.0    1
Name: InvoiceDate, Length: 4372, dtype: category
Categories (5, int64): [1 < 2 < 3 < 4 < 5]

In [37]:
# 合併Series成DataFrame: 方法一
rfm = {'r_score':r_score, 'f_score':f_score, 'm_score':m_score}
rfm_df = pd.DataFrame(rfm)

In [38]:
rfm_df.head()

,r_score,f_score,m_score
CustomerID,,,
12346.0,1,1,1
12347.0,5,1,1
12348.0,5,1,1
12349.0,5,1,1
12350.0,1,1,1


In [24]:
# 合併Series成DataFrame: 方法二
rfm_df = pd.concat([r_score, f_score, m_score], axis=1)
rfm_df.columns = ['r_score', 'f_score', 'm_score']

In [25]:
rfm_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 4372 entries, 12346.0 to 18287.0
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   r_score  4372 non-null   category
 1   f_score  4372 non-null   category
 2   m_score  4372 non-null   category
dtypes: category(3)
memory usage: 47.6 KB


In [26]:
rfm_df = rfm_df.astype(int)

In [27]:
rfm_df.head()

,r_score,f_score,m_score
CustomerID,,,
12346.0,1,1,1
12347.0,5,1,1
12348.0,5,1,1
12349.0,5,1,1
12350.0,1,1,1


In [28]:
rfm_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 4372 entries, 12346.0 to 18287.0
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   r_score  4372 non-null   int64
 1   f_score  4372 non-null   int64
 2   m_score  4372 non-null   int64
dtypes: int64(3)
memory usage: 136.6 KB


## 計算 RFM 總分

兩種方式：

* 直接評估 R, F, M 的重要性，再做加權平均成為一個數字。
* 直接串接 R, F, M 各自的值，秀出一個字串。

In [29]:
# 方法一：加權平均
rfm_df['rfm_score'] = rfm_df['r_score'] * 0.5 + rfm_df['f_score'] * 0.3 + rfm_df['m_score'] * 0.2

In [30]:
# 方法二：RFM组合
rfm_df_tmp = rfm_df.copy()
rfm_df_tmp['r_score'] = rfm_df_tmp['r_score'].astype('str')
rfm_df_tmp['f_score'] = rfm_df_tmp['f_score'].astype('str')
rfm_df_tmp['m_score'] = rfm_df_tmp['m_score'].astype('str')
rfm_df['rfm_comb'] = rfm_df_tmp['r_score']+rfm_df_tmp['f_score']+rfm_df_tmp['m_score']

In [31]:
rfm_df.head()

,r_score,f_score,m_score,rfm_score,rfm_comb
CustomerID,,,,,
12346.0,1,1,1,1.0,111
12347.0,5,1,1,3.0,511
12348.0,5,1,1,3.0,511
12349.0,5,1,1,3.0,511
12350.0,1,1,1,1.0,111


In [32]:
rfm_df.sort_values('rfm_score', ascending=False)

,r_score,f_score,m_score,rfm_score,rfm_comb
CustomerID,,,,,
14911.0,5,4,3,4.3,543
17841.0,5,5,1,4.2,551
14096.0,5,4,2,4.1,542
14646.0,5,2,5,4.1,525
18102.0,5,1,5,3.8,515
...,...,...,...,...,...
12505.0,1,1,1,1.0,111
12651.0,1,1,1,1.0,111
14090.0,1,1,1,1.0,111
